!pip install pulp

!pip install Faker

In [1]:
import itertools

In [2]:
from pulp import *
problem = LpProblem("schedule-opt", LpMaximize)

In [3]:
from faker import Faker
fake = Faker() 

In [4]:
from collections import namedtuple

In [5]:
N_l = 1 # number of teachers
N_g = 1 # number of groups
N_s = 2 # number of courses

In [6]:
L = [fake.name() for _ in range(N_l)]
G = range(0, N_g)
types = ['lection', 'lab', 'practice']
subjs = [fake.job() for _ in range(N_s)]
D = range(0, 2)
T = range(0, 12)

In [7]:
def get_all_sp_keys(L, G, types, subjs, D, T):
    return itertools.product(L, G, types, subjs, D, T)

In [8]:
def get_all_gdt_keys(GG, DD, TT):
    return itertools.product(GG, DD, TT)

In [9]:
def get_all_ldt_keys(LL, DD, TT):
    return itertools.product(LL, DD, TT)

In [10]:
SP = LpVariable.dicts("SP", get_all_sp_keys(L, G, types, subjs, D, T), 0, 1, cat=pulp.LpInteger)

In [11]:
Ig = LpVariable.dicts("Ig", get_all_gdt_keys(G, D, T[:-1]), 0, 1, cat=pulp.LpInteger)

In [12]:
Il = LpVariable.dicts("Il", get_all_ldt_keys(L, D, T[:-1]), 0, 1, cat=pulp.LpInteger)

In [13]:
def get_x_by_gdt(g, d, t, SP):
    s = 0
    for key in itertools.product(L, [g], types, subjs, [d], [t]):
        s += SP[key]
    return s

In [14]:
def get_x_by_ldt(l, d, t, SP):
    s = 0
    for key in itertools.product([l], G, types, subjs, [d], [t]):
        s += SP[key]
    return s

In [15]:
def sum_x_by_g(g, SP):
    sum = 0
    for key in itertools.product(L, [g], types, subjs, D, T):
         sum += SP[key]
    return sum


In [16]:
def sum_x_by_l(l, SP):
    sum = 0
    for key in itertools.product([l], G, types, subjs, D, T):
         sum += SP[key]
    return sum

In [17]:
alpha = 1.0
beta = 1.0
gamma = 1.0

In [18]:
def get_c(key):
    return 1.0

# Problem

In [19]:
all_lectures = [alpha * get_c(key) * SP[key] for key in get_all_sp_keys(L, G, types, subjs, D, T)]

In [20]:
all_groups_no_gaps = [beta * Ig[key] for key in get_all_gdt_keys(G, D, T[:-1])]

In [21]:
all_teachers_no_gaps = [gamma * Il[key] for key in get_all_ldt_keys(L, D, T[:-1])]

In [22]:
problem += lpSum(all_lectures + all_groups_no_gaps + all_teachers_no_gaps)

# Constraints

## Minimize gaps between lessons for teacher

In [23]:
for (teacher, day, time) in itertools.product(L, D, T[:-1]):
    problem += get_x_by_ldt(teacher, day, time, SP) + get_x_by_ldt(teacher, day, time + 1, SP) -1 >= Il[teacher, day, time]

## Minimize gaps between lessons for group

In [24]:
for (group, day, time) in itertools.product(G, D, T[:-1]):
    problem += get_x_by_gdt(group, day, time, SP) + get_x_by_gdt(group, day, time + 1, SP) -1 >= Ig[group, day, time]

## Amount of lessons for group must be no more than payload in plan

In [25]:
for g in G:
    problem += sum_x_by_g(g, SP) <= len(D) * 3

## Amount of lessons for teacher must be no more than payload in plan

In [26]:
for teacher in L:
    problem += sum_x_by_l(teacher, SP) <= len(D) * 3

## One lesson per time slot for one group

In [27]:
def sum_x_for_group_at_dt(g, d, t, SP):
    s = 0
    for key in itertools.product(L, [g], types, subjs, [d], [t]):
        s += SP[key]
    return s

In [28]:
for key in get_all_gdt_keys(G, D, T):
    s = sum_x_for_group_at_dt(*key, SP)
    problem += s <= 1
    problem += s >= 0

## One lesson per time slot for one teacher

In [29]:
def sum_x_for_teacher_at_dt(l, d, t, SP):
    s = 0
    for key in itertools.product([l], G, types, subjs, [d], [t]):
        s += SP[key]
    return s

In [30]:
for key in get_all_ldt_keys(L, D, T):
    s = sum_x_for_teacher_at_dt(*key, SP)
    problem += s <= 1
    problem += s >= 0

## No more than 3 lessons for one group per day

In [31]:
def sum_x_for_group_per_day(g, d, SP):
    s = 0
    for key in itertools.product(L, [g], types, subjs, [d], T):
        s += SP[key]
    return s

In [32]:
for (group, day) in itertools.product(G, D):
    problem += sum_x_for_group_per_day(group, day, SP) <= 3

## No more than 4 lessons for one teacher per day

In [33]:
def sum_x_for_teacher_per_day(l, d, SP):
    s = 0
    for key in itertools.product([l], G, types, subjs, [d], T):
        s += SP[key]
    return s

In [34]:
for (teacher, day) in itertools.product(L, D):
    problem += sum_x_for_teacher_per_day(teacher, day, SP) <= 4

## No more than 2 lessons of one subject for group per day

In [35]:
def sum_x_of_same_subject_for_group_per_day(g, d, subj, SP):
    s = 0
    for key in itertools.product(L, [g], types, [subj], [d], T):
        s += SP[key]
    return s

In [36]:
for (group, day, subj) in itertools.product(G, D, subjs):
    problem += sum_x_of_same_subject_for_group_per_day(group, day, subj, SP) <= 2

# Solution

In [37]:
solvers.GLPK_CMD().available()

'C:\\Users\\Zver\\Anaconda3\\Library\\bin\\glpsol.exe'

In [38]:
problem.solve()

-1

In [39]:
def print_solution(problem):
    print("Status:", LpStatus[problem.status])
    for v in problem.variables():
         if (v.varValue > 0.0):
            print(v.name, "=", v.varValue)
            #print("Total Cost =", pulp.value(problem.objective))

In [40]:
print_solution(problem)

Status: Infeasible
Ig_(0,_0,_1) = 1.0
Ig_(0,_0,_3) = 1.0
Ig_(0,_0,_9) = 1.0
Il_('Debra_Henderson',_0,_0) = 1.0
Il_('Debra_Henderson',_0,_1) = 1.0
Il_('Debra_Henderson',_0,_2) = 1.0
Il_('Debra_Henderson',_0,_3) = 1.0
Il_('Debra_Henderson',_0,_4) = 1.0
Il_('Debra_Henderson',_0,_5) = 1.0
Il_('Debra_Henderson',_0,_6) = 1.0
Il_('Debra_Henderson',_0,_7) = 1.0
Il_('Debra_Henderson',_0,_8) = 1.0
Il_('Debra_Henderson',_0,_9) = 1.0
Il_('Debra_Henderson',_1,_0) = 1.0
Il_('Debra_Henderson',_1,_1) = 1.0
Il_('Debra_Henderson',_1,_10) = 1.0
Il_('Debra_Henderson',_1,_2) = 1.0
Il_('Debra_Henderson',_1,_3) = 1.0
Il_('Debra_Henderson',_1,_6) = 1.0
Il_('Debra_Henderson',_1,_7) = 1.0
Il_('Debra_Henderson',_1,_8) = 1.0
Il_('Debra_Henderson',_1,_9) = 1.0
SP_('Debra_Henderson',_0,_'lab',_'Administrator,_education',_1,_1) = 1.0
SP_('Debra_Henderson',_0,_'lab',_'Administrator,_education',_1,_3) = 1.0
SP_('Debra_Henderson',_0,_'lab',_'Electronics_engineer',_0,_1) = 1.0
SP_('Debra_Henderson',_0,_'lab',_'Electroni

In [41]:
SP[('Dustin Coleman',0 , 'practice', 'Pharmacist, community', 0, 1)] = 0.0
SP[('Dustin Coleman',0 , 'practice', 'Pharmacist, community', 0, 2)] = 1.0
Ig[(0, 0, 2)] = 1.0

In [42]:
print(value(problem.objective))

6.0


In [43]:
#print([get_x_by_gdt(0, 0, time, SP) for time in T])
print([SP[('Christopher Terrell', 0, type_,'Electronics engineer', 0, time)] for (type_, time) in itertools.product(types, T)])
#print(SP)

KeyError: ('Christopher Terrell', 0, 'lection', 'Electronics engineer', 0, 0)

In [ ]:
pulp.pulpTestAll()